In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:

# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
Nasa=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomonasa_2_dataset.csv",header=None,index_col=0)
columns_Nasa=['ProjName','CatofApp','ForG','Center','YearofDev','DevMode','rely','data','cplx','time','stor','virt','turn','acap','aexp','pcap','vexp','lexp','modp','tool','sced','loc','Effort']
Nasa.set_axis(columns_Nasa,axis='columns',inplace=True)
Nasa.rename_axis("Features", axis=1,inplace=True)
Nasa.rename_axis("Projects ID", axis=0,inplace=True)
Nasa.reset_index(inplace=True)
Nasa.drop(columns=['Projects ID'], inplace=True)

#Nasa dataset preprocessing
def NASA_quantifier(x):
    if x=='vl':
        return 0
    elif x=='l':
        return 1
    elif x=='n':
        return 2
    elif x=='h':
        return 3
    elif x=='vh':
        return 4
    elif x=='xh':
        return 5
    else:
        return x

Nasa=Nasa.applymap(NASA_quantifier)
dataset = Nasa

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort'])  # Features
y = dataset['Effort'].values  # Target (effort)
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)


# Step 3: Feature Selection using Harmony Search
def objective_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    cross_val_rmse = []
    
    X_selected = X.iloc[:, selected_features]
    X_selected = pd.get_dummies(X_selected)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=0)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
        
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)
#====================================================================================================================
# Harmony Search Parameters
num_features = X.shape[1]
hms = 20  # Harmony memory size
iterations = 100  # Number of iterations

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < 0.5:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj < current_obj:
        harmony_memory[rnd_choice] = new_harmony

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print(selected_features)
#======================================================================================================================

#if len(selected_features) == 0:
#   return float('-inf')  # Penalize subsets with no selected features
    
    
# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
        'neurons_input': (10, 50),
        'neurons_hidden': (10, 50),
        'num_hidden_layers': (1, 5),
        'learning_rate': (1e-5, 1e-2),
        'batch_size': (8,32 ),
        'epochs': (5, 20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)
    
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
X_selected = X.iloc[:, selected_features]
X_selected = pd.get_dummies(X_selected)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


...

1/1 [==============================] - 0s 47ms/step
| 32        | -484.9    | 14.85     | 13.56     | 0.007422  | 46.53     | 13.17     | 3.828     |
1/1 [==============================] - 0s 37ms/step
| 33        | -573.4    | 20.87     | 19.84     | 0.006639  | 10.14     | 30.0      | 2.181     |
1/1 [==============================] - 0s 31ms/step
| 34        | -452.2    | 9.572     | 9.772     | 0.004336  | 49.95     | 12.57     | 3.785     |
1/1 [==============================] - 0s 50ms/step
| 35        | -462.2    | 20.82     | 12.09     | 0.0002202 | 33.14     | 35.18     | 3.528     |
1/1 [==============================] - 0s 31ms/step
| 36        | -428.9    | 21.55     | 13.64     | 0.007056  | 13.22     | 15.76     | 1.572     |
1/1 [==============================] - 0s 47ms/step
| 37        | -408.1    | 27.8      | 6.016     | 0.009573  | 23.59     | 11.92     | 3.756     |
1/1 [==============================] - 0s 31ms/step
| 38        | -462.0    | 22.29     | 5.297     

1/1 [==============================] - 0s 31ms/step
| 29        | -633.5    | 16.25     | 6.382     | 0.006976  | 27.18     | 14.36     | 2.119     |
1/1 [==============================] - 0s 52ms/step
| 30        | -660.7    | 9.128     | 18.03     | 0.001733  | 38.56     | 11.6      | 4.9       |
1/1 [==============================] - 0s 27ms/step
| 31        | -716.9    | 27.88     | 15.54     | 0.001176  | 26.81     | 12.74     | 1.15      |
1/1 [==============================] - 0s 47ms/step
| 32        | -625.0    | 14.85     | 13.56     | 0.007422  | 46.53     | 13.17     | 3.828     |
1/1 [==============================] - 0s 33ms/step
| 33        | -589.6    | 20.87     | 19.84     | 0.006639  | 10.14     | 30.0      | 2.181     |
1/1 [==============================] - 0s 32ms/step
| 34        | -713.9    | 9.572     | 9.772     | 0.004336  | 49.95     | 12.57     | 3.785     |
1/1 [==============================] - 0s 47ms/step
| 35        | -718.1    | 20.82     | 12.09     

1/1 [==============================] - 0s 31ms/step
| 26        | -1.449e+0 | 15.14     | 19.39     | 0.007448  | 14.38     | 33.71     | 3.899     |
1/1 [==============================] - 0s 40ms/step
| 27        | -1.54e+03 | 8.0       | 17.49     | 0.01      | 12.24     | 33.47     | 1.0       |
1/1 [==============================] - 0s 32ms/step
| 28        | -1.341e+0 | 8.293     | 20.0      | 0.01      | 33.69     | 16.28     | 2.141     |
1/1 [==============================] - 0s 47ms/step
| 29        | -1.426e+0 | 19.32     | 20.0      | 0.01      | 10.0      | 43.89     | 5.0       |
1/1 [==============================] - 0s 31ms/step
| 30        | -1.857e+0 | 20.31     | 5.086     | 0.01      | 33.2      | 49.41     | 5.0       |
1/1 [==============================] - 0s 49ms/step
| 31        | -1.503e+0 | 23.13     | 20.0      | 0.01      | 15.58     | 41.34     | 5.0       |
1/1 [==============================] - 0s 31ms/step
| 32        | -1.371e+0 | 8.0       | 20.0      

1/1 [==============================] - 0s 47ms/step
| 23        | -1.222e+0 | 10.23     | 20.0      | 0.01      | 41.41     | 18.09     | 5.0       |
1/1 [==============================] - 0s 36ms/step
| 24        | -1.302e+0 | 8.0       | 20.0      | 0.01      | 42.66     | 20.09     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 25        | -1.31e+03 | 13.55     | 20.0      | 0.01      | 44.42     | 18.81     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 26        | -1.338e+0 | 14.16     | 20.0      | 0.01      | 39.91     | 19.74     | 5.0       |
1/1 [==============================] - 0s 32ms/step
| 27        | -1.514e+0 | 11.01     | 19.92     | 0.008873  | 42.37     | 18.09     | 1.087     |
1/1 [==============================] - 0s 48ms/step
| 28        | -1.247e+0 | 19.34     | 20.0      | 0.01      | 43.9      | 16.77     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 29        | -1.334e+0 | 20.54     | 20.0      

1/1 [==============================] - 0s 31ms/step
| 20        | -653.4    | 29.01     | 10.02     | 0.003366  | 47.63     | 31.9      | 1.897     |
1/1 [==============================] - 0s 38ms/step
| 21        | -576.5    | 18.53     | 19.72     | 0.002655  | 30.99     | 21.28     | 3.285     |
1/1 [==============================] - 0s 30ms/step
| 22        | -629.1    | 10.85     | 10.28     | 0.007454  | 37.26     | 32.57     | 2.429     |
1/1 [==============================] - 0s 31ms/step
| 23        | -592.5    | 16.34     | 15.16     | 0.004878  | 13.67     | 10.61     | 1.883     |
1/1 [==============================] - 0s 31ms/step
| 24        | -542.3    | 23.59     | 18.05     | 0.005597  | 23.23     | 48.02     | 3.229     |
1/1 [==============================] - 0s 41ms/step
| 25        | -626.5    | 20.57     | 7.095     | 0.004777  | 31.34     | 14.44     | 2.192     |
1/1 [==============================] - 0s 32ms/step
| 26        | -661.1    | 13.42     | 6.584     

In [3]:
selected_features

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1])